In [1]:
# set up
import os
import numpy as np
import pandas as pd
import functools as func
import functions_v2 as f
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
home_path = "C:\\Users\joche\FIM Kernkompetenzzentrum\Projekt VIdES - Dokumente\General\\07_Arbeitsordner\\04_Ergebnisse\\20220920_Ergebnisse_4_Wochen"
os.chdir(home_path)